### Example Usage: Build Scaling Relation with the Traditional Method
TODO: format of this is very messy

#### Workflow

##### Stage One: Preparation

1.1 Prepare Adsorption Energy Data

1.2 Decide (Two) Descriptors and Group Members

1.3 Provide Reaction and Free-Species Energies

##### Stage Two: Build

2. Run the Relation Builder (and Check Metrics)

##### Stage Three: Analysis (Visualization)

3. Visualize Results with Volcano Plots and More

## Stage One: Preparation


### 1.1 Adsorption Energy

Expect adsorption energy data as `pandas.DataFrame`,
and you could easily generate one from a local `csv` file,
where the row header should be adsorbate names, 
and the column header being sample names, like the following:
```sql
+-----------+------+------+-----+-------+------+------+
|           | *CO2 |*COOH | ... |*OCH3  | *O   | *OH  |
+-----------+------+------+-----+-------+------+------+
| Cu@g-C3N4 | 0.89 | 4.37 | ... | 3.98  | -1.73| 0.17 |
| Ni@C2N    |-4.57 |-4.95 | ... |-0.93  | -2.81|-3.21 |
| ...       | ...  | ...  | ... | ...   | ...  | ...  |
| Pt@SiO2   |-2.36 | 3.69 | ... | 3.12  | 0.29 | 4.84 |
| Au@Al2O3  | 2.15 |-2.35 | ... | 1.36  | 1.07 | 4.56 |
+-----------+------+------+-----+-------+------+------+

```

In [6]:
# TODO: need to fix this
import sys
sys.path.append('..')

# Imports
from scaling.data import Eads, Reaction
from scaling.relation import Builder, EadsRelation, DeltaERelation

In [2]:
# Initialize Eads from csv file
eads = Eads.from_csv("g-C3N4.csv")

### 1.2: Decide Descriptors and Group Members

As of the traditional method, you would need to nominate
two descriptors and their own group members, for example
the following example for CO<sub>2</sub> to CH<sub>4</sub> reduction reaction:

```python
groups = {
    "*CO": ["*COOH", "*CHO", "*CH2O"],  # C-centered group
    "*OH": ["*OCH3", "*O"]              # O-Centered group
}
```

In [3]:
# Set "groups"
eads.groups = {
    "*CO": ["*COOH", "*CHO", "*CH2O"],  # C-centered group
    "*OH": ["*OCH3", "*O"]              # O-Centered group
}

### 1.3: Define Reaction and Free-Species Energies

To convert an adsorption energy scaling Relation (`EadsRelation`) to a
reaction energy Relation (`DeltaERelation`), the reaction pathway
and species energies are necessary.

The `Reaction` class provides a `from_str` method to generate a Reaction
from a string, which should take the following form:
```
*CO2 + H+ + e- -> *COOH
*COOH + H+ + e- -> *CO + H2O_l
```

Add an energy dict is also needed:
```
energy_dict = {
    "CO2": (X, Y),
    "COOH": (X, Y),
    "CO": (X, Y),
    "H+": (X, Y),
    "e-": (0, 0),
    "H2O_l": (X, Y),
}
```


In [4]:
reaction_str = """
*CO2 + H++e- -> *COOH
*COOH + H++e- -> *CO + H2O_l
"""

# TODO: use proper energy and correction
energy_dict = {
    "CO2": (0, 0),
    "COOH": (0, 0),
    "CO": (0, 0),
    "H++e-": (0, 0),
    "H2O_l": (0, 0),
}

reaction = Reaction.from_str(reaction_str, energy_dict)

/Users/yang/developer/scaling/examples/../scaling/data/species.py:84: UserWarning: Non-negative energy found.
  warnings.warn("Non-negative energy found.")


## Stage Two: Build

In [7]:
### 2.1 Build adsorption energy scaling relation (EadsRelation)

builder = Builder(data=eads)

eadsrelation = builder.build_traditional(eads.groups)


KeyError: '*CO'

In [ ]:
### 2.2 Convert EadsRelation to reaction energy Relation (DeltaERelation)